### Demo SQL Analyst Agent — Powered by Gemini 2.5 Flash

In this notebook, we build a small Agentic AI SQL Analyst using:

Gemini 2.5 Flash (LLM brain)
LangChain SQL Agent Toolkit
SQLite (dummy in-code database)

The agent can understand natural language queries → generate SQL → execute → and return results.

In [1]:
#Install Required Libraries
!pip install langchain-google-genai langchain-community google-generativeai ipywidgets


Defaulting to user installation because normal site-packages is not writeable
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.4-py3-none-any.whl.metadata (4.2 kB)
  Using cached google_generativeai-0.8.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.7.2-py3-none-any.whl.metadata (4.0 kB)
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
  Using cached google_generativeai-0.7.1-py3-none-any.whl.metadata (3.9 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-classic 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.80 which is incompatible.
langchain-classic 1.0.0 requires langchain-text-splitters<2.0.0,>=1.0.0, but you have langchain-text-splitters 0.3.11 which is incompatible.
langchain-openai 1.0.3 requires langchain-core<2.0.0,>=1.0.2, but you have langchain-core 0.3.80 which is incompatible.
langgraph-prebuilt 1.0.4 requires langchain-core>=1.0.0, but you have langchain-core 0.3.80 which is incompatible.


### Setup: Configure API Key

In [4]:
import os

os.environ["GOOGLE_API_KEY"] = "YOUR_REAL_KEY_HERE"

if os.environ["GOOGLE_API_KEY"] is None:
    print("Please set your GOOGLE_API_KEY environment variable before continuing.")
else:
    print("API Key Loaded Successfully")

API Key Loaded Successfully


### Perception — Creating a Dummy SQLite Database

In [6]:
import sqlite3

# Create a local SQLite database
db_path = "company_data.db"
connection = sqlite3.connect(db_path)
cursor = connection.cursor()

# Reset the table if exists
cursor.execute("DROP TABLE IF EXISTS employees")

# Create table
cursor.execute("""
CREATE TABLE employees (
    id INTEGER,
    name TEXT,
    department TEXT,
    salary INTEGER
)
""")

# Insert sample data
data = [
    (1, "Alice", "Sales", 50000),
    (2, "Bob", "Engineering", 120000),
    (3, "Charlie", "Sales", 55000),
    (4, "David", "HR", 40000),
    (5, "Eve", "Engineering", 125000),
]

cursor.executemany("INSERT INTO employees VALUES (?, ?, ?, ?)", data)
connection.commit()
connection.close()

print("SQLite Database Ready!")

SQLite Database Ready!


### Reasoning — Loading the Gemini 2.5 Flash LLM

In [7]:
from langchain_google_genai import (ChatGoogleGenerativeAI, HarmBlockThreshold, HarmCategory)

# Safety settings (allow SQL generation)
safety_settings = {
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
}

# Load the LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    safety_settings=safety_settings,
    google_api_key=os.environ["GOOGLE_API_KEY"]
)

print("LLM Ready (Gemini 2.5 Flash)")

LLM Ready (Gemini 2.5 Flash)


### Action — Creating the SQL Agent

In [18]:
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit, create_sql_agent

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    handle_parsing_errors=True,
)

print("SQL Agent Ready!")

SQL Agent Ready!


### Autonomy Loop — Ask the Agent Questions

In [20]:
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML
import time, io, contextlib

# Styling for chat + logs
style = """
<style>
.chat-container {
    font-family: Arial, sans-serif;
    line-height: 1.5;
}

.bubble-user {
    background-color: #007bff;
    color: white;
    padding: 10px 14px;
    border-radius: 12px;
    margin: 6px 0;
    max-width: 60%;
    float: right;
    clear: both;
}

.bubble-agent {
    background-color: #e6e6e6;
    color: #000;
    padding: 10px 14px;
    border-radius: 12px;
    margin: 6px 0;
    max-width: 60%;
    float: left;
    clear: both;
}

.typing {
    font-style: italic;
    opacity: 0.6;
    padding: 4px;
}

.log-box {
    background: #111;
    color: #0f0;
    padding: 10px;
    font-family: monospace;
    border-radius: 8px;
    height: 260px;
    overflow-y: auto;
    white-space: pre-wrap;
    margin-top: 20px;
    border: 2px solid #333;
}
</style>
"""

display(HTML(style))

# UI widgets
chat_area = widgets.HTML(value="<div class='chat-container'></div>")
thinking_log = widgets.HTML(value="<div class='log-box'>🧠 Agent Thinking Log...\n</div>")

input_box = widgets.Text(
    placeholder='Ask something...',
    description='You:',
    layout=widgets.Layout(width="600px")
)
send_btn = widgets.Button(
    description='Send',
    button_style='primary',
    layout=widgets.Layout(width='120px')
)

buttons = widgets.HBox([send_btn])

# Internal storage
chat_log = ""
log_text = "🧠 Agent Thinking Log...\n"

def update_chat():
    chat_area.value = f"<div class='chat-container'>{chat_log}</div>"

def update_log():
    thinking_log.value = f"<div class='log-box'>{log_text}</div>"

def show_typing():
    global chat_log
    chat_log += "<div class='typing'>🤖 Agent is thinking...</div>"
    update_chat()

def remove_typing():
    global chat_log
    chat_log = chat_log.replace("<div class='typing'>🤖 Agent is thinking...</div>", "")
    update_chat()


def on_send_clicked(_):
    global chat_log, log_text

    user_msg = input_box.value.strip()
    if user_msg == "":
        return

    # Add user bubble
    chat_log += f"<div class='bubble-user'>{user_msg}</div>"
    update_chat()
    input_box.value = ""

    if user_msg.lower() in ["exit", "quit", "bye"]:
        chat_log += "<div class='bubble-agent'>Goodbye!</div>"
        update_chat()
        return

    show_typing()
    update_chat()

    # Capture printed logs (stdout)
    log_capture = io.StringIO()
    with contextlib.redirect_stdout(log_capture):
        try:
            response = agent_executor.invoke({"input": user_msg})
            answer = response["output"]
        except Exception as e:
            answer = f"❌ Error: {str(e)}"

    remove_typing()

    # Get captured logs
    raw_logs = log_capture.getvalue()
    if raw_logs.strip() == "":
        raw_logs = "(No logs printed — verbose may be disabled)\n"

    # Append logs
    log_text += f"\n\n=== USER ASKED: {user_msg} ===\n"
    log_text += raw_logs
    log_text += f"\nFINAL ANSWER: {answer}\n"
    update_log()

    # Add agent bubble
    chat_log += f"<div class='bubble-agent'>{answer}</div>"
    update_chat()


send_btn.on_click(on_send_clicked)

# Display UI
display(Markdown("## SQL Agent — Chat Interface"))
display(chat_area)
display(input_box)
display(buttons)
display(Markdown("## Agent Thinking Log"))
display(thinking_log)

## SQL Agent — Chat Interface

HTML(value="<div class='chat-container'></div>")

Text(value='', description='You:', layout=Layout(width='600px'), placeholder='Ask something...')

## Agent Thinking Log

HTML(value="<div class='log-box'>🧠 Agent Thinking Log...\n</div>")